# ISIC Pipeline - Grad-CAM Compatible Version (10 Epochs Demo)

Bu notebook, Grad-CAM ile uyumlu model mimarisi kullanır.

## Temel Fark

**Orijinal:** `preprocess_input` model içinde
```python
x = keras.applications.mobilenet_v2.preprocess_input(inputs)  # Model içinde
x = base_model(x)
```

**Fix'li:** `preprocess_input` veri pipeline'ında
```python
# Model sadece base_model kullanır
x = base_model(inputs)

# Preprocessing veri yüklemede:
dataset = dataset.map(lambda img, label: (
    keras.applications.mobilenet_v2.preprocess_input(img), label
))
```

## Demo Ayarları

- **EPOCHS:** 10 (hızlı test için)
- **Amaç:** Grad-CAM'in çalıştığını doğrulamak

In [1]:
# Install dependencies
!pip install -q kagglehub tensorflow matplotlib scikit-learn pandas tf-keras-vis

/bin/bash: line 1: pip: command not found


In [2]:
# Download dataset
import kagglehub
dataset_path = kagglehub.dataset_download('nodoubttome/skin-cancer9-classesisic')
print(f"Dataset downloaded to: {dataset_path}")

100%|██████████████████████████████████████████████████████████████████████████████████████████| 786M/786M [00:28<00:00, 28.6MB/s]

Extracting files...


Dataset downloaded to: /home/burak/.cache/kagglehub/datasets/nodoubttome/skin-cancer9-classesisic/versions/1


In [3]:
# Config
IMG_SIZE = (224, 224)
IMG_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 10  # Demo: hızlı test
SEED = 42

import tensorflow as tf
tf.random.set_seed(SEED)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

2026-02-02 04:37:29.486360: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.20.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Load and analyze dataset
import os
import pandas as pd

data_dir = os.path.join(dataset_path, 'skin-cancer9-classesisic')

# Find all images
image_data = []
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_data.append({
                'path': os.path.join(class_path, img_file),
                'class': class_name
            })

df = pd.DataFrame(image_data)
class_counts = df['class'].value_counts()
print(f"\nTotal images: {len(df)}")
print(f"\nClass distribution:\n{class_counts}")

# Select top 2 classes
top_classes = class_counts.head(2).index.tolist()
df_filtered = df[df['class'].isin(top_classes)].copy()
print(f"\nSelected classes: {top_classes}")
print(f"Filtered dataset size: {len(df_filtered)}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/burak/.cache/kagglehub/datasets/nodoubttome/skin-cancer9-classesisic/versions/1/skin-cancer9-classesisic'

In [ ]:
# Create train/val/test split
from sklearn.model_selection import train_test_split
import numpy as np

# Encode labels
class_to_idx = {cls: idx for idx, cls in enumerate(top_classes)}
df_filtered['label'] = df_filtered['class'].map(class_to_idx)

# Split: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(
    df_filtered, test_size=0.3, stratify=df_filtered['label'], random_state=SEED
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df['label'], random_state=SEED
)

print(f"Train: {len(train_df)}")
print(f"Val: {len(val_df)}")
print(f"Test: {len(test_df)}")

# Extract arrays
train_paths = train_df['path'].values
train_labels = train_df['label'].values
val_paths = val_df['path'].values
val_labels = val_df['label'].values
test_paths = test_df['path'].values
test_labels = test_df['label'].values

class_names = tuple(top_classes)

In [ ]:
# Data loading functions
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0  # Normalize to [0, 1]
    return img, label

# Augmentation
def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, 0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    return img, label

# Create datasets WITHOUT preprocessing (will add per-model)
def create_dataset(paths, labels, batch_size, augment_data=False, preprocessing_fn=None):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Apply model-specific preprocessing if provided
    if preprocessing_fn is not None:
        def apply_preprocessing(img, label):
            # Preprocessing expects [0, 255] range
            img = img * 255.0
            img = preprocessing_fn(img)
            return img, label
        ds = ds.map(apply_preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
    
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
# Upload models_fixed.py
!echo "Upload 'models_fixed.py' file to /content/"

# Verify
!ls -lh /content/models_fixed.py

In [ ]:
# Import fixed models
from models_fixed import (
    create_scratch_cnn_fixed,
    create_mobilenet_fixed,
    create_efficientnet_fixed,
    unfreeze_and_recompile
)

print("✅ Fixed models imported")

## Train Scratch CNN

In [ ]:
# Create datasets for Scratch CNN (no preprocessing)
train_ds_scratch = create_dataset(train_paths, train_labels, BATCH_SIZE, augment_data=True)
val_ds_scratch = create_dataset(val_paths, val_labels, BATCH_SIZE)

# Create model
scratch_model = create_scratch_cnn_fixed(IMG_SHAPE)
scratch_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)

# Callbacks
callbacks_scratch = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Train
print("🚀 Training Scratch CNN...")
history_scratch = scratch_model.fit(
    train_ds_scratch,
    validation_data=val_ds_scratch,
    epochs=EPOCHS,
    callbacks=callbacks_scratch
)

print("✅ Scratch CNN training complete")

## Train MobileNetV2 (Fixed)

In [ ]:
# Create datasets WITH MobileNet preprocessing
mobilenet_preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

train_ds_mobilenet = create_dataset(
    train_paths, train_labels, BATCH_SIZE, 
    augment_data=True, preprocessing_fn=mobilenet_preprocess
)
val_ds_mobilenet = create_dataset(
    val_paths, val_labels, BATCH_SIZE,
    preprocessing_fn=mobilenet_preprocess
)

# Create model (preprocessing external!)
mobilenet_model = create_mobilenet_fixed(IMG_SHAPE, trainable=False)

callbacks_mobilenet = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Train
print("🚀 Training MobileNetV2 (Fixed)...")
history_mobilenet = mobilenet_model.fit(
    train_ds_mobilenet,
    validation_data=val_ds_mobilenet,
    epochs=EPOCHS,
    callbacks=callbacks_mobilenet
)

print("✅ MobileNetV2 training complete")

## Train EfficientNetB0 (Fixed)

In [ ]:
# Create datasets WITH EfficientNet preprocessing
efficientnet_preprocess = tf.keras.applications.efficientnet.preprocess_input

train_ds_efficient = create_dataset(
    train_paths, train_labels, BATCH_SIZE,
    augment_data=True, preprocessing_fn=efficientnet_preprocess
)
val_ds_efficient = create_dataset(
    val_paths, val_labels, BATCH_SIZE,
    preprocessing_fn=efficientnet_preprocess
)

# Create model (preprocessing external!)
efficientnet_model = create_efficientnet_fixed(IMG_SHAPE, trainable=False)

callbacks_efficient = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Train
print("🚀 Training EfficientNetB0 (Fixed)...")
history_efficient = efficientnet_model.fit(
    train_ds_efficient,
    validation_data=val_ds_efficient,
    epochs=EPOCHS,
    callbacks=callbacks_efficient
)

print("✅ EfficientNetB0 training complete")

## Test Grad-CAM

In [ ]:
# Import standard Grad-CAM (should work now!)
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
import matplotlib.pyplot as plt

def test_gradcam(model, model_name, img_path, preprocessing_fn=None):
    """Test if Grad-CAM works on the fixed model."""
    print(f"\nTesting Grad-CAM on {model_name}...")
    
    try:
        # Load image
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img_resized = tf.image.resize(img, IMG_SIZE)
        original = img_resized.numpy() / 255.0
        
        # Preprocess
        if preprocessing_fn:
            prep = preprocessing_fn(img_resized.numpy())
        else:
            prep = img_resized.numpy() / 255.0
        prep = np.expand_dims(prep, 0)
        
        # Create Gradcam
        gradcam = Gradcam(model, model_modifier=ReplaceToLinear(), clone=False)
        
        def score_fn(output):
            return output[:, 0]
        
        # Generate
        cam = gradcam(score_fn, prep, penultimate_layer=-1)
        heatmap = cam[0]
        
        # Resize
        heatmap_resized = tf.image.resize(
            heatmap[..., np.newaxis], IMG_SIZE
        ).numpy().squeeze()
        
        if heatmap_resized.max() > 0:
            heatmap_resized /= heatmap_resized.max()
        
        # Visualize
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        axes[0].imshow(original)
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        axes[1].imshow(heatmap_resized, cmap='jet')
        axes[1].set_title('Grad-CAM')
        axes[1].axis('off')
        
        overlay = 0.6 * original + 0.4 * plt.cm.jet(heatmap_resized)[:, :, :3]
        axes[2].imshow(np.clip(overlay, 0, 1))
        axes[2].set_title('Overlay')
        axes[2].axis('off')
        
        plt.suptitle(f'{model_name} - Grad-CAM Test ✅')
        plt.tight_layout()
        plt.show()
        
        print(f"  ✅ SUCCESS! Grad-CAM works for {model_name}")
        return True
        
    except Exception as e:
        print(f"  ❌ FAILED: {e}")
        import traceback
        traceback.print_exc()
        return False

# Test on one image from each model
test_image = test_paths[0]

print("="*70)
print("🔬 GRAD-CAM COMPATIBILITY TEST")
print("="*70)

results = {
    'Scratch CNN': test_gradcam(scratch_model, 'Scratch CNN', test_image),
    'MobileNetV2': test_gradcam(mobilenet_model, 'MobileNetV2', test_image, mobilenet_preprocess),
    'EfficientNetB0': test_gradcam(efficientnet_model, 'EfficientNetB0', test_image, efficientnet_preprocess)
}

print("\n" + "="*70)
print("📊 RESULTS")
print("="*70)
for model_name, success in results.items():
    icon = "✅" if success else "❌"
    print(f"  {icon} {model_name}")

if all(results.values()):
    print("\n🎉 ALL MODELS GRAD-CAM COMPATIBLE!")
    print("\n✅ Fix başarılı! Artık tüm modellere Grad-CAM uygulayabilirsiniz.")
else:
    print("\n⚠️  Some models failed. Check errors above.")

print("="*70)

## Summary

Bu notebook:
1. ✅ Modelleri fix'li mimari ile eğitti (10 epoch)
2. ✅ Grad-CAM uyumluluğunu test etti

**Eğer test başarılıysa:**
- Bu modelleri 100 epoch ile yeniden eğitin
- Grad-CAM görsellerini oluşturun
- outputs.zip'e ekleyin

**Eğer test başarısızsa:**
- Manuel Grad-CAM script'i deneyin
- Ya da README ile pragmatik çözümü kullanın